In [1]:
import tensorflow as tf

In [32]:
def residual(x, filters):
  filter1, filter2, filter3 = filters

  #layer1
  y = tf.keras.layers.Conv2D(filters=filter1, kernel_size=1)(x)
  y = tf.keras.layers.BatchNormalization(axis=3)(y)
  y = tf.keras.layers.Activation('relu')(y)

  #layer2
  y = tf.keras.layers.Conv2D(filters=filter2, kernel_size=3, padding='same')(y)
  y = tf.keras.layers.BatchNormalization(axis=3)(y)
  y = tf.keras.layers.Activation('relu')(y)

  #layer3
  y = tf.keras.layers.Conv2D(filters=filter3, kernel_size=1)(y)
  y = tf.keras.layers.BatchNormalization(axis=3)(y)
  y = tf.keras.layers.Add()([x, y])

  y = tf.keras.layers.Activation('relu')(y)

  return y


In [33]:
def residual_block(filters):
  return lambda x: residual(x, filters)

In [29]:
def conv(x, filters, strides=(2,2)):
  filter1, filter2, filter3 = filters

  y = tf.keras.layers.Conv2D(filters=filter1, kernel_size = 1, strides=strides)(x)
  y = tf.keras.layers.BatchNormalization(axis=3)(y)
  y = tf.keras.layers.Activation('relu')(y)

  y = tf.keras.layers.Conv2D(filters=filter1, kernel_size = 3, padding='same')(y)
  y = tf.keras.layers.BatchNormalization(axis=3)(y)
  y = tf.keras.layers.Activation('relu')(y)

  y = tf.keras.layers.Conv2D(filters=filter3, kernel_size=1)(y)
  y = tf.keras.layers.BatchNormalization(axis=3)(y)

  shortcut = tf.keras.layers.Conv2D(filters=filter3, kernel_size=1, strides=strides)(x)
  shortcut = tf.keras.layers.BatchNormalization(axis=3)(shortcut)

  y = tf.keras.layers.add([y, shortcut])
  y = tf.keras.layers.Activation('relu')(y)

  return y


In [30]:
def conv_block(filters, strides=(2, 2)):
  return lambda x: conv(x, filters, strides)

In [34]:
#ResNet 50

CLASSES = [i for i in range(1000)]

x = tf.keras.layers.Input(shape=[224, 224, 3])

y = tf.keras.layers.ZeroPadding2D(padding=3)(x)
y = tf.keras.layers.Conv2D(filters=64, kernel_size=7, padding='valid')(y)
y = tf.keras.layers.BatchNormalization(axis=3)(y)
y = tf.keras.layers.Activation('relu')(y)
y = tf.keras.layers.ZeroPadding2D(padding=1)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=3, strides=(2, 2))(y)

y = conv_block([64, 64, 256], strides=(1,1))(y)
y = residual_block([64, 64, 256])(y)
y = residual_block([64, 64, 256])(y)

y = conv_block([128, 128, 512], strides=(2,2))(y)
y = residual_block([128, 128, 512])(y)
y = residual_block([128, 128, 512])(y)
y = residual_block([128, 128, 512])(y)

y = conv_block([256, 256, 1024], strides=(2,2))(y)
y = residual_block([256, 256, 1024])(y)
y = residual_block([256, 256, 1024])(y)
y = residual_block([256, 256, 1024])(y)

y = conv_block([512, 512, 2048], strides=(2,2))(y)
y = residual_block([512, 512, 2048])(y)
y = residual_block([512, 512, 2048])(y)

y = tf.keras.layers.GlobalAveragePooling2D()(y)
y = tf.keras.layers.Dense(len(CLASSES), activation='softmax')(y)

model = tf.keras.Model(x, y)

In [35]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d_18 (ZeroPad  (None, 230, 230, 3)          0         ['input_11[0][0]']            
 ding2D)                                                                                          
                                                                                                  
 conv2d_22 (Conv2D)          (None, 224, 224, 64)         9472      ['zero_padding2d_18[0][0]']   
                                                                                                  
 batch_normalization_21 (Ba  (None, 224, 224, 64)         256       ['conv2d_22[0][0]']       